In [25]:
from typing import Tuple
import os

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

## Dataset

In [18]:
class CovDataset(Dataset):

    def __init__(self, X_path, y_path, transforms=None,
                 n_windows=115, window_size=36) -> None:
        super().__init__()

        self.X = np.genfromtxt(X_path, delimiter=',').reshape(-1, n_windows, window_size)
        self.y = np.genfromtxt(y_path, delimiter=',')
        assert self.X.shape[0] == self.y.shape[0]

        self.transforms = transforms
        
    def __getitem__(self, index) -> Tuple[np.ndarray, np.ndarray]:
        if self.transforms is not None:
            return self.transforms(self.X[index]), self.y[index]
        else:
            return self.X[index], self.y[index]
    
    def __len__(self) -> int:
        return len(self.X)

## Transforms

In [7]:
class Jittering(object):

    def __init__(self, std) -> None:
        self.std = std

    def __call__(self, obj) -> object:
        noise = np.random.normal(loc=0, scale=self.std, size=obj.shape)
        return obj + noise

## Loading data 

In [19]:
cwd = os.getcwd()

X_train_path = os.path.join(cwd, 'data', 'pack1', 'converted', 'listen-noise', 'A_train.csv')
y_train_path = os.path.join(cwd, 'data', 'pack1', 'converted', 'listen-noise', 'y_train.csv')

X_val_path = os.path.join(cwd, 'data', 'pack1', 'converted', 'listen-noise', 'A_val.csv')
y_val_path = os.path.join(cwd, 'data', 'pack1', 'converted', 'listen-noise', 'y_val.csv')

X_test_path = os.path.join(cwd, 'data', 'pack1', 'converted', 'listen-noise', 'A_test.csv')
y_test_path = os.path.join(cwd, 'data', 'pack1', 'converted', 'listen-noise', 'y_test.csv')

# creating datasets
tr = transforms.Compose([Jittering(0.1)])

train_dataset = CovDataset(X_train_path, y_train_path, transforms=tr)
val_dataset = CovDataset(X_val_path, y_val_path, transforms=None)
test_dataset = CovDataset(X_test_path, y_test_path, transforms=None)

In [20]:
# dataloaders

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

## Train and Evaluate

In [61]:
def evaluate(model, device, data_loader, loss_function=nn.BCELoss(reduction='sum')) -> Tuple[float, float]:
    model.eval()
    loss = 0
    correct = 0
    length = 0
    
    with torch.no_grad():
        for batch in data_loader:

            X, y = batch[0].float().to(device), batch[1].float().to(device)
            output = model(X).squeeze()
            
            loss += loss_function(output, y).item() 
            pred = (output > 0.5).float()
            correct += (pred == y).sum().item()
            length += batch[1].shape[0]

    return loss / length, correct / length


def train(model, device, train_loader, val_loader, test_loader, n_epoch, optimizer, scheduler, 
          writer, loss_function=nn.BCELoss(reduction='sum'), max_norm=None, track_gradient=False,
          initial_epoch=1, global_step=0):

    model.to(device)
    
    for epoch in range(initial_epoch, n_epoch + initial_epoch):
        model.train()

        for batch in train_loader:

            global_step += 1
            X, y = batch[0].float().to(device), batch[1].float().to(device)
            
            optimizer.zero_grad()
            output = model(X).squeeze()
            loss = loss_function(output, y)
            loss.backward()
            
            if max_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
                
            if track_gradient:
                norm = 0
                for p in model.parameters():
                    norm += (p.grad.data.detach().norm(2) ** 2).item()
                norm = norm ** 0.5
                writer.add_scalar('gradient_norm', norm, global_step)

            optimizer.step()
            scheduler.step()
            
        train_loss, train_acc = evaluate(model, device, train_loader)
        val_loss, val_acc = evaluate(model, device, val_loader)
        test_loss, test_acc = evaluate(model, device, test_loader)

        print(f'Epoch {epoch}|\t train_loss: {train_loss:.3f};\t train_acc: {train_acc:.3f}')

        writer.add_scalars('loss', {
            'train': train_loss,
            'val': val_loss,
            'test': test_loss
        }, epoch)
        writer.add_scalars('accuracy', {
            'train': train_acc,
            'val': val_acc,
            'test': test_acc
        }, epoch)

        writer.flush()

    return n_epoch + initial_epoch, global_step

## LSTM Model

In [65]:
class LSTM_Net(nn.Module):

    def __init__(self, input_size, hidden_size, 
                 activation=nn.ReLU(),  dropout_p=0.2) -> None:
        super().__init__()
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers=3, batch_first=True, dropout=dropout_p)
        self.clf = nn.Sequential(
            nn.BatchNorm1d(hidden_size),
            nn.Linear(hidden_size, 256), nn.Dropout(dropout_p), activation,
            nn.Linear(256, 128), nn.Dropout(dropout_p), activation,
            nn.Linear(128, 64), nn.Dropout(dropout_p), activation,
            nn.Linear(64, 1), nn.Sigmoid()
        )

    def forward(self, X):
        
        output, _, = self.rnn(X)
        output = output[:, -1, :]  # (batch, hidden_size)
        
        output = self.clf(output)
        return output

In [66]:
log_dir = os.path.join('.', 'runs', '3-Layer-LSTM-fixDropout')
writer = SummaryWriter(log_dir=log_dir)

In [67]:
# creating model
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = LSTM_Net(input_size=36, hidden_size=256)
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

In [68]:
last_epoch, last_step = train(
    model, device, train_loader, val_loader, test_loader, 50, optimizer, scheduler, writer, track_gradient=True
)

Epoch 1|	 train_loss: 0.693;	 train_acc: 0.525
Epoch 2|	 train_loss: 0.691;	 train_acc: 0.569
Epoch 3|	 train_loss: 0.684;	 train_acc: 0.589
Epoch 4|	 train_loss: 0.664;	 train_acc: 0.612
Epoch 5|	 train_loss: 0.642;	 train_acc: 0.619
Epoch 6|	 train_loss: 0.611;	 train_acc: 0.646
Epoch 7|	 train_loss: 0.587;	 train_acc: 0.682
Epoch 8|	 train_loss: 0.564;	 train_acc: 0.686
Epoch 9|	 train_loss: 0.539;	 train_acc: 0.707
Epoch 10|	 train_loss: 0.517;	 train_acc: 0.717
Epoch 11|	 train_loss: 0.501;	 train_acc: 0.731
Epoch 12|	 train_loss: 0.486;	 train_acc: 0.742
Epoch 13|	 train_loss: 0.476;	 train_acc: 0.750
Epoch 14|	 train_loss: 0.470;	 train_acc: 0.751
Epoch 15|	 train_loss: 0.462;	 train_acc: 0.751
Epoch 16|	 train_loss: 0.457;	 train_acc: 0.761
Epoch 17|	 train_loss: 0.453;	 train_acc: 0.757
Epoch 18|	 train_loss: 0.449;	 train_acc: 0.762
Epoch 19|	 train_loss: 0.446;	 train_acc: 0.764
Epoch 20|	 train_loss: 0.445;	 train_acc: 0.763
Epoch 21|	 train_loss: 0.442;	 train_acc: 0.766
E